# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: 
Date: 

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [10]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('../data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

ALPHAVANTAGE_API_KEY loaded? True


## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [8]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_INTRADAY','symbol':SYMBOL,'outputsize':'compact','interval':'60min','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    print(js)
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','4. close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','Adj Close']]
    df_api.columns = ['date','adj_close']

v_api = validate(df_api, ['date','adj_close']); v_api

{'Meta Data': {'1. Information': 'Intraday (60min) open, high, low, close prices and volume', '2. Symbol': 'AAPL', '3. Last Refreshed': '2025-08-20 19:00:00', '4. Interval': '60min', '5. Output Size': 'Compact', '6. Time Zone': 'US/Eastern'}, 'Time Series (60min)': {'2025-08-20 19:00:00': {'1. open': '225.6500', '2. high': '225.7000', '3. low': '225.5000', '4. close': '225.6390', '5. volume': '48903'}, '2025-08-20 18:00:00': {'1. open': '225.7000', '2. high': '225.7800', '3. low': '225.5800', '4. close': '225.6400', '5. volume': '34223'}, '2025-08-20 17:00:00': {'1. open': '225.8800', '2. high': '225.9000', '3. low': '216.9087', '4. close': '225.6900', '5. volume': '34827'}, '2025-08-20 16:00:00': {'1. open': '225.9700', '2. high': '241.3268', '3. low': '216.8995', '4. close': '225.9000', '5. volume': '13832271'}, '2025-08-20 15:00:00': {'1. open': '226.1000', '2. high': '227.1300', '3. low': '225.9200', '4. close': '225.9600', '5. volume': '6728347'}, '2025-08-20 14:00:00': {'1. open'

{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [11]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved ../data/raw/api_source-alpha_symbol-AAPL_20250820-214653.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [12]:
import re

SCRAPE_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
resp = requests.get(SCRAPE_URL, timeout=30, headers={"User-Agent": "data-ingestion-homework/1.0"})
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

def norm_header(s: str) -> str:
    s = re.sub(r"\[[^\]]*\]", "", s)            
    s = s.replace("\xa0", " ").strip().lower()
    s = re.sub(r"[^a-z0-9]+", "_", s)           
    return s.strip("_")


candidates = []
t = soup.find("table", id="constituents")
if t: candidates.append(t)
candidates += soup.find_all("table", class_="wikitable")

def table_to_df(table):
    head = table.find("tr")
    if not head: return None
    cols = [norm_header(th.get_text(" ", strip=True)) for th in head.find_all("th")]
    if not cols: return None
    # we want the actual constituents table
    needed = {"symbol", "gics_sector"}
    if not needed.issubset(set(cols)):
        return None

    rows = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all(["td","th"])
        if not tds: continue
        vals = [td.get_text(" ", strip=True) for td in tds]
        # align to header length
        if len(vals) < len(cols):
            vals += [""] * (len(cols) - len(vals))
        elif len(vals) > len(cols):
            vals = vals[:len(cols)]
        rows.append(dict(zip(cols, vals)))
    if not rows:
        return None
    return pd.DataFrame(rows)

df_scrape = None
for tbl in candidates:
    df = table_to_df(tbl)
    if df is not None:
        df_scrape = df
        break

if df_scrape is None:
    raise RuntimeError("Could not find S&P 500 constituents table with required columns.")

# Standardize common column names if Wikipedia label shifts slightly
rename_map = {}
cols = set(df_scrape.columns)
if "security" not in cols:
    for alt in ["company", "constituent", "name"]:
        if alt in cols:
            rename_map[alt] = "security"
            break
if rename_map:
    df_scrape = df_scrape.rename(columns=rename_map)

# Light typing / cleaning
if "date_first_added" in df_scrape.columns:
    df_scrape["date_first_added"] = pd.to_datetime(df_scrape["date_first_added"], errors="coerce")
if "cik" in df_scrape.columns:
    df_scrape["cik"] = df_scrape["cik"].astype(str).str.replace(r"\.0$", "", regex=True)

df_scrape.head(), list(df_scrape.columns)

v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape


{'missing': [], 'shape': (503, 8), 'na_total': 0}

In [14]:
_ = save_csv(df_scrape, prefix='scrape', site='wikipedia', table='S&P 500 component stocks')


Saved ../data/raw/scrape_site-wikipedia_table-S&P 500 component stocks_20250820-215336.csv


## Sources
- API: Alpha Vantage (TIME_SERIES_INTRADAY) if `ALPHA_VANTAGE_KEY` present; otherwise yfinance.
- Params: {'function':'TIME_SERIES_INTRADAY','symbol':SYMBOL,'outputsize':'compact','interval':'60min','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
- Ticker: AAPL
- Scrape: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies (constituents table)


## Validation
     Build a list of any required columns that are NOT present in df
     Return a small summary:
     - 'missing': which required columns are absent
     - 'shape': (num_rows, num_cols)
     - 'na_total': total number of NA/NaN/None cells across the ENTIRE DataFrame

## Reproducible Filenames
- `api_source-alpha_symbol-AAPL_20250820-214653.csv`
- `scrape_site-wikipedia_table-S&P 500 component stocks_20250820-215336.csv`


## Assumptions & Risks
- Wikipedia table structure may change.
- Alpha Vantage has rate limits; yfinance used as a no-key fallback.
